# Historical tweet counts, by term

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import altair as alt
import altair_stiles as altstiles

In [3]:
alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [4]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

---

## Search for a term

In [5]:
term = "Elon Musk"

#### Get daily counts

In [6]:
!twarc2 counts --granularity 'day' '{term}' --archive --csv data/raw/{term.replace(' ', '_')}.csv

100%|████████| Processed 16 years/16 years [03:46<00:00, 58087262 tweets total ]


In [7]:
src = pd.read_csv(f"data/raw/{term.replace(' ', '_')}.csv")

In [8]:
src.head()

,start,end,day_count
0,2022-10-06T00:00:00.000Z,2022-10-07T00:00:00.000Z,143849
1,2022-10-07T00:00:00.000Z,2022-10-08T00:00:00.000Z,87022
2,2022-10-08T00:00:00.000Z,2022-10-09T00:00:00.000Z,84562
3,2022-10-09T00:00:00.000Z,2022-10-10T00:00:00.000Z,60451
4,2022-10-10T00:00:00.000Z,2022-10-11T00:00:00.000Z,86252


#### Clean up dates

In [9]:
src["year"] = pd.to_datetime(src["start"]).dt.strftime("%Y")
src["month_year"] = pd.to_datetime(src["start"]).dt.strftime("%Y-%m")
src["date"] = pd.to_datetime(src["start"]).dt.strftime("%Y-%m-%d")

In [10]:
src[src["date"] == src["date"].max()]

,start,end,day_count,year,month_year,date
30,2022-11-05T00:00:00.000Z,2022-11-05T20:31:34.000Z,521174,2022,2022-11,2022-11-05


In [11]:
src_grouped = (
    src.groupby(["date", "year", "month_year"]).agg({"day_count": sum}).reset_index()
)

#### Slim down and re-order the dataframe

In [12]:
df = (
    src_grouped[["year", "month_year", "date", "day_count"]]
    .sort_values("date", ascending=False)
    .copy()
)

In [13]:
df.head()

,year,month_year,date,day_count
6073,2022,2022-11,2022-11-05,521174
6072,2022,2022-11,2022-11-04,520654
6071,2022,2022-11,2022-11-03,386748
6070,2022,2022-11,2022-11-02,405299
6069,2022,2022-11,2022-11-01,430302


#### When was the first mention? 

In [14]:
first = df[df["day_count"] > 0]["date"].tail(1).iloc[0]
first

'2007-03-21'

#### How many total mentions? 

In [15]:
df.day_count.sum()

58087262

#### Average mentions? 

In [16]:
df.day_count.mean()

9563.26341784656

#### Make a new dataframe starting from first mention

In [17]:
df_complete = df[df["date"] >= first]

#### Which day was mentioned the most? 

In [18]:
df_complete[df_complete["day_count"] == df_complete["day_count"].max()]

,year,month_year,date,day_count
5880,2022,2022-04,2022-04-26,2390196


In [19]:
df_complete.sort_values("day_count", ascending=False).head(10)

,year,month_year,date,day_count
5880,2022,2022-04,2022-04-26,2390196
5879,2022,2022-04,2022-04-25,1636662
6065,2022,2022-10,2022-10-28,1573279
5881,2022,2022-04,2022-04-27,1185765
5868,2022,2022-04,2022-04-14,745391
5869,2022,2022-04,2022-04-15,728448
6066,2022,2022-10,2022-10-29,722162
5882,2022,2022-04,2022-04-28,590675
6068,2022,2022-10,2022-10-31,554652
6073,2022,2022-11,2022-11-05,521174


#### Chart it

In [20]:
alt.Chart(df_complete[df_complete["date"] > "2010-01-01"]).mark_bar(
    color="#1DA1F2", size=1
).encode(
    x=alt.X("date:T", axis=alt.Axis(format="%b. %Y", tickCount=5), title=""),
    y=alt.Y("day_count:Q", title=" ", axis=alt.Axis(tickCount=4, format="0,M")),
).properties(
    width=650, height=200, title=f"Daily mentions of the '{term}' on Twitter"
)

alt.Chart(...)

---

## Aggregate 

#### Groupby month/year

In [21]:
months = (
    df_complete.groupby(["month_year"])
    .agg({"day_count": sum})
    .sort_values("month_year", ascending=False)
    .reset_index()
)

#### Chart it

In [22]:
alt.Chart(months).mark_bar(color="#1DA1F2", size=3).encode(
    x=alt.X("month_year:T", axis=alt.Axis(format="%b. %Y", tickCount=10), title=""),
    y=alt.Y("day_count:Q", title=" ", axis=alt.Axis(tickCount=5, format="0,M")),
).properties(width=900, height=350, title=f"Monthly mentions of the {term} on Twitter")

alt.Chart(...)

## Exports

In [23]:
months.to_csv(
    f"data/processed/twitter_mentions_{term.replace(' ', '')}_months.csv",
    index=False,
)
df_complete.to_csv(
    f"data/processed/twitter_mentions_{term.replace(' ', '')}_days.csv",
    index=False,
)

In [24]:
months.day_count.sum()

58087262

In [25]:
months.day_count.mean()

307340.0105820106